### **MODEL BUILDING AND EVALUATION**

In [1]:
# Importing required modules
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import xgboost as xgb
from imblearn.over_sampling import SMOTE

from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score, classification_report